In [50]:
import findspark
findspark.init()

import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, array, lit, monotonically_increasing_id 

spark = SparkSession.builder.master("local[8]").appName("Examen_2_M2_DCDD").getOrCreate()

sc = spark.sparkContext

import pandas as pd
pd.set_option("display.max_columns",200)

In [2]:
sc

<SparkContext master=local[8] appName=Examen_2_M2_DCDD>

In [3]:
spark

In [4]:
import os
path = '../../../data/modulo2/examen/dataexam2/'
os.listdir(path)

['Credit Cards Transactions.zip',
 'credit_card_transactions-ibm_v2.csv',
 'sd254_cards.csv',
 'sd254_users.csv',
 'User0_credit_card_transactions.csv',
 'balanced_tx.csv']

In [5]:
#clients = spark.read.csv(path + 'sd254_users.csv')
clients = spark.read.format('csv').option('header','true').load(path + 'sd254_users.csv')

In [6]:
clients.describe()

DataFrame[summary: string, id: string, Person: string, Current Age: string, Retirement Age: string, Birth Year: string, Birth Month: string, Gender: string, Address: string, Apartment: string, City: string, State: string, Zipcode: string, Latitude: string, Longitude: string, Per Capita Income - Zipcode: string, Yearly Income - Person: string, Total Debt: string, FICO Score: string, Num Credit Cards: string]

In [7]:
clients.head()

Row(id='0', Person='Hazel Robinson', Current Age='53', Retirement Age='66', Birth Year='1966', Birth Month='11', Gender='Female', Address='462 Rose Lane', Apartment=None, City='La Verne', State='CA', Zipcode='91750', Latitude='34.15', Longitude='-117.76', Per Capita Income - Zipcode='$29278', Yearly Income - Person='$59696', Total Debt='$127613', FICO Score='787', Num Credit Cards='5')

In [8]:
# cards = spark.read.csv(path + 'sd254_cards.csv')
cards = spark.read.format('csv').option('header', 'true').load(path + 'sd254_cards.csv')

In [9]:
cards.head()

Row(User='0', CARD INDEX='0', Card Brand='Visa', Card Type='Debit', Card Number='4344676511950444', Expires='12/2022', CVV='623', Has Chip='YES', Cards Issued='2', Credit Limit='$24295', Acct Open Date='09/2002', Year PIN last Changed='2008', Card on Dark Web='No')

In [10]:
# tx = spark.read.csv(path + 'credit_card_transactions-ibm_v2.csv')
tx = spark.read.format('csv').option('header', 'true').load(path + 'credit_card_transactions-ibm_v2.csv')

In [11]:
tx.head()

Row(User='0', Card='0', Year='2002', Month='9', Day='1', Time='06:21', Amount='$134.09', Use Chip='Swipe Transaction', Merchant Name='3527213246127876953', Merchant City='La Verne', Merchant State='CA', Zip='91750.0', MCC='5300', Errors?=None, Is Fraud?='No')

In [12]:
clients.take(5)

[Row(id='0', Person='Hazel Robinson', Current Age='53', Retirement Age='66', Birth Year='1966', Birth Month='11', Gender='Female', Address='462 Rose Lane', Apartment=None, City='La Verne', State='CA', Zipcode='91750', Latitude='34.15', Longitude='-117.76', Per Capita Income - Zipcode='$29278', Yearly Income - Person='$59696', Total Debt='$127613', FICO Score='787', Num Credit Cards='5'),
 Row(id='1', Person='Sasha Sadr', Current Age='53', Retirement Age='68', Birth Year='1966', Birth Month='12', Gender='Female', Address='3606 Federal Boulevard', Apartment=None, City='Little Neck', State='NY', Zipcode='11363', Latitude='40.76', Longitude='-73.74', Per Capita Income - Zipcode='$37891', Yearly Income - Person='$77254', Total Debt='$191349', FICO Score='701', Num Credit Cards='5'),
 Row(id='2', Person='Saanvi Lee', Current Age='81', Retirement Age='67', Birth Year='1938', Birth Month='11', Gender='Female', Address='766 Third Drive', Apartment=None, City='West Covina', State='CA', Zipcode='

In [13]:
cards.take(5)

[Row(User='0', CARD INDEX='0', Card Brand='Visa', Card Type='Debit', Card Number='4344676511950444', Expires='12/2022', CVV='623', Has Chip='YES', Cards Issued='2', Credit Limit='$24295', Acct Open Date='09/2002', Year PIN last Changed='2008', Card on Dark Web='No'),
 Row(User='0', CARD INDEX='1', Card Brand='Visa', Card Type='Debit', Card Number='4956965974959986', Expires='12/2020', CVV='393', Has Chip='YES', Cards Issued='2', Credit Limit='$21968', Acct Open Date='04/2014', Year PIN last Changed='2014', Card on Dark Web='No'),
 Row(User='0', CARD INDEX='2', Card Brand='Visa', Card Type='Debit', Card Number='4582313478255491', Expires='02/2024', CVV='719', Has Chip='YES', Cards Issued='2', Credit Limit='$46414', Acct Open Date='07/2003', Year PIN last Changed='2004', Card on Dark Web='No'),
 Row(User='0', CARD INDEX='3', Card Brand='Visa', Card Type='Credit', Card Number='4879494103069057', Expires='08/2024', CVV='693', Has Chip='NO', Cards Issued='1', Credit Limit='$12400', Acct Ope

In [14]:
tx.take(5)

[Row(User='0', Card='0', Year='2002', Month='9', Day='1', Time='06:21', Amount='$134.09', Use Chip='Swipe Transaction', Merchant Name='3527213246127876953', Merchant City='La Verne', Merchant State='CA', Zip='91750.0', MCC='5300', Errors?=None, Is Fraud?='No'),
 Row(User='0', Card='0', Year='2002', Month='9', Day='1', Time='06:42', Amount='$38.48', Use Chip='Swipe Transaction', Merchant Name='-727612092139916043', Merchant City='Monterey Park', Merchant State='CA', Zip='91754.0', MCC='5411', Errors?=None, Is Fraud?='No'),
 Row(User='0', Card='0', Year='2002', Month='9', Day='2', Time='06:22', Amount='$120.34', Use Chip='Swipe Transaction', Merchant Name='-727612092139916043', Merchant City='Monterey Park', Merchant State='CA', Zip='91754.0', MCC='5411', Errors?=None, Is Fraud?='No'),
 Row(User='0', Card='0', Year='2002', Month='9', Day='2', Time='17:45', Amount='$128.95', Use Chip='Swipe Transaction', Merchant Name='3414527459579106770', Merchant City='Monterey Park', Merchant State='C

In [15]:
clients.show(5)

+---+--------------+-----------+--------------+----------+-----------+------+--------------------+---------+-------------+-----+-------+--------+---------+---------------------------+----------------------+----------+----------+----------------+
| id|        Person|Current Age|Retirement Age|Birth Year|Birth Month|Gender|             Address|Apartment|         City|State|Zipcode|Latitude|Longitude|Per Capita Income - Zipcode|Yearly Income - Person|Total Debt|FICO Score|Num Credit Cards|
+---+--------------+-----------+--------------+----------+-----------+------+--------------------+---------+-------------+-----+-------+--------+---------+---------------------------+----------------------+----------+----------+----------------+
|  0|Hazel Robinson|         53|            66|      1966|         11|Female|       462 Rose Lane|     null|     La Verne|   CA|  91750|   34.15|  -117.76|                     $29278|                $59696|   $127613|       787|               5|
|  1|    Sasha S

In [16]:
cards.show(5)

+----+----------+----------+---------------+----------------+-------+---+--------+------------+------------+--------------+---------------------+----------------+
|User|CARD INDEX|Card Brand|      Card Type|     Card Number|Expires|CVV|Has Chip|Cards Issued|Credit Limit|Acct Open Date|Year PIN last Changed|Card on Dark Web|
+----+----------+----------+---------------+----------------+-------+---+--------+------------+------------+--------------+---------------------+----------------+
|   0|         0|      Visa|          Debit|4344676511950444|12/2022|623|     YES|           2|      $24295|       09/2002|                 2008|              No|
|   0|         1|      Visa|          Debit|4956965974959986|12/2020|393|     YES|           2|      $21968|       04/2014|                 2014|              No|
|   0|         2|      Visa|          Debit|4582313478255491|02/2024|719|     YES|           2|      $46414|       07/2003|                 2004|              No|
|   0|         3|     

In [17]:
tx.show(5)

+----+----+----+-----+---+-----+-------+-----------------+-------------------+-------------+--------------+-------+----+-------+---------+
|User|Card|Year|Month|Day| Time| Amount|         Use Chip|      Merchant Name|Merchant City|Merchant State|    Zip| MCC|Errors?|Is Fraud?|
+----+----+----+-----+---+-----+-------+-----------------+-------------------+-------------+--------------+-------+----+-------+---------+
|   0|   0|2002|    9|  1|06:21|$134.09|Swipe Transaction|3527213246127876953|     La Verne|            CA|91750.0|5300|   null|       No|
|   0|   0|2002|    9|  1|06:42| $38.48|Swipe Transaction|-727612092139916043|Monterey Park|            CA|91754.0|5411|   null|       No|
|   0|   0|2002|    9|  2|06:22|$120.34|Swipe Transaction|-727612092139916043|Monterey Park|            CA|91754.0|5411|   null|       No|
|   0|   0|2002|    9|  2|17:45|$128.95|Swipe Transaction|3414527459579106770|Monterey Park|            CA|91754.0|5651|   null|       No|
|   0|   0|2002|    9|  3|0

In [18]:
clients.count()

2000

In [19]:
tx.printSchema()

root
 |-- User: string (nullable = true)
 |-- Card: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- Use Chip: string (nullable = true)
 |-- Merchant Name: string (nullable = true)
 |-- Merchant City: string (nullable = true)
 |-- Merchant State: string (nullable = true)
 |-- Zip: string (nullable = true)
 |-- MCC: string (nullable = true)
 |-- Errors?: string (nullable = true)
 |-- Is Fraud?: string (nullable = true)



In [20]:
cards.printSchema()

root
 |-- User: string (nullable = true)
 |-- CARD INDEX: string (nullable = true)
 |-- Card Brand: string (nullable = true)
 |-- Card Type: string (nullable = true)
 |-- Card Number: string (nullable = true)
 |-- Expires: string (nullable = true)
 |-- CVV: string (nullable = true)
 |-- Has Chip: string (nullable = true)
 |-- Cards Issued: string (nullable = true)
 |-- Credit Limit: string (nullable = true)
 |-- Acct Open Date: string (nullable = true)
 |-- Year PIN last Changed: string (nullable = true)
 |-- Card on Dark Web: string (nullable = true)



In [21]:
clients.printSchema()

root
 |-- id: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Current Age: string (nullable = true)
 |-- Retirement Age: string (nullable = true)
 |-- Birth Year: string (nullable = true)
 |-- Birth Month: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Apartment: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Per Capita Income - Zipcode: string (nullable = true)
 |-- Yearly Income - Person: string (nullable = true)
 |-- Total Debt: string (nullable = true)
 |-- FICO Score: string (nullable = true)
 |-- Num Credit Cards: string (nullable = true)



In [22]:
clients.show(3, vertical=True)

-RECORD 0-------------------------------------------
 id                          | 0                    
 Person                      | Hazel Robinson       
 Current Age                 | 53                   
 Retirement Age              | 66                   
 Birth Year                  | 1966                 
 Birth Month                 | 11                   
 Gender                      | Female               
 Address                     | 462 Rose Lane        
 Apartment                   | null                 
 City                        | La Verne             
 State                       | CA                   
 Zipcode                     | 91750                
 Latitude                    | 34.15                
 Longitude                   | -117.76              
 Per Capita Income - Zipcode | $29278               
 Yearly Income - Person      | $59696               
 Total Debt                  | $127613              
 FICO Score                  | 787            

In [23]:
cards.show(10, vertical=True)

-RECORD 0---------------------------------
 User                  | 0                
 CARD INDEX            | 0                
 Card Brand            | Visa             
 Card Type             | Debit            
 Card Number           | 4344676511950444 
 Expires               | 12/2022          
 CVV                   | 623              
 Has Chip              | YES              
 Cards Issued          | 2                
 Credit Limit          | $24295           
 Acct Open Date        | 09/2002          
 Year PIN last Changed | 2008             
 Card on Dark Web      | No               
-RECORD 1---------------------------------
 User                  | 0                
 CARD INDEX            | 1                
 Card Brand            | Visa             
 Card Type             | Debit            
 Card Number           | 4956965974959986 
 Expires               | 12/2020          
 CVV                   | 393              
 Has Chip              | YES              
 Cards Issu

In [24]:
tx.show(3, vertical=True)

-RECORD 0-----------------------------
 User           | 0                   
 Card           | 0                   
 Year           | 2002                
 Month          | 9                   
 Day            | 1                   
 Time           | 06:21               
 Amount         | $134.09             
 Use Chip       | Swipe Transaction   
 Merchant Name  | 3527213246127876953 
 Merchant City  | La Verne            
 Merchant State | CA                  
 Zip            | 91750.0             
 MCC            | 5300                
 Errors?        | null                
 Is Fraud?      | No                  
-RECORD 1-----------------------------
 User           | 0                   
 Card           | 0                   
 Year           | 2002                
 Month          | 9                   
 Day            | 1                   
 Time           | 06:42               
 Amount         | $38.48              
 Use Chip       | Swipe Transaction   
 Merchant Name  | -727612

In [25]:
tx_cards = tx.join(cards, tx['User'] == cards['User'],'right')

In [26]:
tx_cards.count()

89828418

In [27]:
tx_cards.printSchema()

root
 |-- User: string (nullable = true)
 |-- Card: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Month: string (nullable = true)
 |-- Day: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Amount: string (nullable = true)
 |-- Use Chip: string (nullable = true)
 |-- Merchant Name: string (nullable = true)
 |-- Merchant City: string (nullable = true)
 |-- Merchant State: string (nullable = true)
 |-- Zip: string (nullable = true)
 |-- MCC: string (nullable = true)
 |-- Errors?: string (nullable = true)
 |-- Is Fraud?: string (nullable = true)
 |-- User: string (nullable = true)
 |-- CARD INDEX: string (nullable = true)
 |-- Card Brand: string (nullable = true)
 |-- Card Type: string (nullable = true)
 |-- Card Number: string (nullable = true)
 |-- Expires: string (nullable = true)
 |-- CVV: string (nullable = true)
 |-- Has Chip: string (nullable = true)
 |-- Cards Issued: string (nullable = true)
 |-- Credit Limit: string (nullable = true)
 |-- Acct

In [28]:
tx = tx.withColumnRenamed('Is Fraud?', 'tgt')

In [29]:
tx.groupBy('tgt').count().show()

+---+--------+
|tgt|   count|
+---+--------+
| No|24357143|
|Yes|   29757|
+---+--------+



In [30]:
29757/24357143

0.00122169500749739

$x = \frac{29,757*100}{40} = 74,392$

Por lo que un dataFrame con proporción $60/40$ quedaría de tamaño 74,392 y se deben extraer 44,635 registros para 'Yes'

Aprox...


In [1]:
major_tx = tx.filter(col('tgt')=='No')
minor_tx = tx.filter(col('tgt')=='Yes')

NameError: name 'tx' is not defined

In [32]:
sampled_majority_tx = major_tx.sample(withReplacement=False, fraction=0.0018, seed=135482)
balanced = sampled_majority_tx.unionAll(minor_tx)

In [33]:
balanced.groupBy('tgt').count().show()

+---+-----+
|tgt|count|
+---+-----+
| No|44219|
|Yes|29757|
+---+-----+



In [34]:
44219 + 29757

73976

In [35]:
44219/73976

0.5977479182437547

In [36]:
29757/73976

0.40225208175624527

In [38]:
cards_tx = cards.join(balanced, (cards['CARD INDEX'] == balanced['Card'])
                      & (cards['User'] == balanced['User']), how='inner').drop(cards['User'])

In [39]:
cards_tx.count()

73976

In [49]:
cards_tx.show(5)

+----------+----------+---------+----------------+-------+---+--------+------------+------------+--------------+---------------------+----------------+----+----+----+-----+---+-----+-------+------------------+--------------------+-------------+--------------+-------+----+-------+---+
|CARD INDEX|Card Brand|Card Type|     Card Number|Expires|CVV|Has Chip|Cards Issued|Credit Limit|Acct Open Date|Year PIN last Changed|Card on Dark Web|User|Card|Year|Month|Day| Time| Amount|          Use Chip|       Merchant Name|Merchant City|Merchant State|    Zip| MCC|Errors?|tgt|
+----------+----------+---------+----------------+-------+---+--------+------------+------------+--------------+---------------------+----------------+----+----+----+-----+---+-----+-------+------------------+--------------------+-------------+--------------+-------+----+-------+---+
|         0|      Visa|    Debit|4344676511950444|12/2022|623|     YES|           2|      $24295|       09/2002|                 2008|           

In [43]:
clients = clients.select("*").withColumn("id_", monotonically_increasing_id())

In [54]:
final_tad = clients.join(cards_tx, (clients['id_'] == cards_tx['User']), how='inner').drop(clients['id_'])

In [55]:
final_tad.count()

73976

In [56]:
df = final_tad.toPandas()

In [57]:
df.head()

,id,Person,Current Age,Retirement Age,Birth Year,Birth Month,Gender,Address,Apartment,City,State,Zipcode,Latitude,Longitude,Per Capita Income - Zipcode,Yearly Income - Person,Total Debt,FICO Score,Num Credit Cards,CARD INDEX,Card Brand,Card Type,Card Number,Expires,CVV,Has Chip,Cards Issued,Credit Limit,Acct Open Date,Year PIN last Changed,Card on Dark Web,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant Name,Merchant City,Merchant State,Zip,MCC,Errors?,tgt
0,0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,None,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,0,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,0,0,2003,6,28,13:19,$10.55,Swipe Transaction,-1605794445852049456,La Verne,CA,91750.0,5812,None,No
1,0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,None,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,0,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,0,0,2003,10,14,20:19,$58.47,Swipe Transaction,-5023497618971072366,Mira Loma,CA,91752.0,5812,None,No
2,0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,None,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,0,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,0,0,2004,8,8,06:15,$129.18,Swipe Transaction,-727612092139916043,Monterey Park,CA,91754.0,5411,None,No
3,0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,None,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,0,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,0,0,2006,9,4,09:55,$48.86,Swipe Transaction,3952145593743244256,La Verne,CA,91750.0,7538,None,No
4,0,Hazel Robinson,53,66,1966,11,Female,462 Rose Lane,None,La Verne,CA,91750,34.15,-117.76,$29278,$59696,$127613,787,5,0,Visa,Debit,4344676511950444,12/2022,623,YES,2,$24295,09/2002,2008,No,0,0,2010,6,10,17:43,$5.29,Online Transaction,-2042049018365856408,ONLINE,None,None,5815,None,No


In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73976 entries, 0 to 73975
Data columns (total 46 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   id                           73976 non-null  object
 1   Person                       73976 non-null  object
 2   Current Age                  73976 non-null  object
 3   Retirement Age               73976 non-null  object
 4   Birth Year                   73976 non-null  object
 5   Birth Month                  73976 non-null  object
 6   Gender                       73976 non-null  object
 7   Address                      73976 non-null  object
 8   Apartment                    19451 non-null  object
 9   City                         73976 non-null  object
 10  State                        73976 non-null  object
 11  Zipcode                      73976 non-null  object
 12  Latitude                     73976 non-null  object
 13  Longitude                    73

In [59]:
final_tad.toPandas().to_csv(path + 'fraud_detection.csv', index=False)